## Timeseries analysis in yt_napari 

`yt_napari` includes a number of ways to load in timeseries data. From a jupyter notebook, you can use `yt_napari.timeseries.add_to_viewer` to specify a set of files to sequentially load and sample and add to an existing `napari.Viewer`. 

Unlike `yt` proper, you first instantiate a selection object **separate** from a dataset object. Right now, the otpions are a `Slice` or 3D `Region`:


In [1]:
from yt_napari import timeseries

In [2]:
timeseries.Slice?

Init signature:
timeseries.Slice(
    field: Tuple[str, str],
    normal: Union[str, int],
    center: Optional[unyt.array.unyt_array] = None,
    width: Optional[unyt.array.unyt_quantity] = None,
    height: Optional[unyt.array.unyt_quantity] = None,
    resolution: Optional[Tuple[int, int]] = (400, 400),
    periodic: Optional[bool] = False,
    take_log: Optional[bool] = None,
)
Docstring:     
Helper class that provides a standard way to create an ABC using
inheritance.
File:           ~/src/yt_general/napari_work/yt-napari/src/yt_napari/timeseries.py
Type:           ABCMeta
Subclasses:     

In [3]:
timeseries.Region?

Init signature:
timeseries.Region(
    field: Tuple[str, str],
    left_edge: Optional[unyt.array.unyt_array] = None,
    right_edge: Optional[unyt.array.unyt_array] = None,
    resolution: Optional[Tuple[int, int, int]] = (400, 400, 400),
    take_log: Optional[bool] = None,
)
Docstring:     
Helper class that provides a standard way to create an ABC using
inheritance.
File:           ~/src/yt_general/napari_work/yt-napari/src/yt_napari/timeseries.py
Type:           ABCMeta
Subclasses:     

Once you create a `Slice` or `Region`, you can pass that to `add_to_viewer` and it will be used to sample each timestep specified. 

## Slices through a timeseries

To start, let's initialize a `napari` viewer

In [4]:
import napari 

In [5]:
v = napari.Viewer()

and let's build the `Slice` object that will get applied to each timestep. To do so, you need to at least specify the field to sample and the normal axis for the slice:

In [6]:
s = timeseries.Slice(('enzo', 'Density'), 'x')                   

### loading as a stack

In [11]:
file_pattern = "enzo_tiny_cosmology/DD????/DD????"

In [18]:
%%time
%%capture
yt.set_log_level(40)
timeseries.add_to_viewer(v,s,file_pattern=file_pattern, load_as_stack=True);

CPU times: user 33.2 s, sys: 508 ms, total: 33.7 s
Wall time: 34.3 s


### loading a range of matches 

In the case above, we are loading in 47 timesteps. We can also specify a `file_range` tuple in the form of `(start, stop, step)`, and the identified files will be subsampled from the full range of matched files. To extract every 10th, for example:


In [20]:
%%time
%%capture
timeseries.add_to_viewer(v,s,file_pattern=file_pattern, file_range=(0, 50, 10), load_as_stack=True);

CPU times: user 3.58 s, sys: 56 ms, total: 3.63 s
Wall time: 3.59 s


### loading specific timesteps 

And finally, you can specify the exact files you want with `file_list`. If you also provide a `file_dir`, it will get pre-prended to the filenames in `file_list` to save you some typing:

In [7]:
%%time
%%capture
flist = ["DD0024/DD0024", "DD0034/DD0034", "DD0041/DD0041"]
timeseries.add_to_viewer(v,s, file_dir="enzo_tiny_cosmology", file_list=flist, load_as_stack=True);

yt : [INFO     ] 2023-08-10 16:56:49,036 Parameters: current_time              = 120.81665329669
yt : [INFO     ] 2023-08-10 16:56:49,037 Parameters: domain_dimensions         = [32 32 32]
yt : [INFO     ] 2023-08-10 16:56:49,037 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2023-08-10 16:56:49,038 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2023-08-10 16:56:49,039 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2023-08-10 16:56:49,039 Parameters: current_redshift          = 0.73734987821014
yt : [INFO     ] 2023-08-10 16:56:49,040 Parameters: omega_lambda              = 0.727
yt : [INFO     ] 2023-08-10 16:56:49,040 Parameters: omega_matter              = 0.273
yt : [INFO     ] 2023-08-10 16:56:49,040 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2023-08-10 16:56:49,041 Parameters: hubble_constant           = 0.702
yt : [INFO     ] 2023-08-10 16:56:49,045 Making a fixed resolution buffer of (('enzo', 'Density')) 4

## loading delayed arrays

### spin up a dask client 


### multithreading and yt

Note that `yt` is not gauranteed to be thread safe. In practice, there are some situations where it **might** be and for this particular dataset, it actually is as long as logging is disabled (which is enforced within `add_to_viewer` when `use_dask=True`. 

In [7]:
from dask.distributed import Client 

In [8]:
c = Client(n_workers=5, threads_per_worker=5)

/home/chavlin/.pyenv/versions/3.9.16/envs/ytnapari/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39127 instead
  warnings.warn(


In [9]:
c

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:39127/status,
Dashboard: http://127.0.0.1:39127/status,Workers: 5
Total threads: 25,Total memory: 31.18 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34955,Workers: 5
Dashboard: http://127.0.0.1:39127/status,Total threads: 25
Started: Just now,Total memory: 31.18 GiB
Comm: tcp://127.0.0.1:46225,Total threads: 5
Dashboard: http://127.0.0.1:44559/status,Memory: 6.24 GiB
Nanny: tcp://127.0.0.1:34697,


Best, to use with `load_as_stack` and to supply contrast limits:

In [10]:
%%capture
timeseries.add_to_viewer(v, s, file_pattern=file_pattern, load_as_stack=True, 
                         use_dask=True, 
                         contrast_limits=(-1, 2))

Parsing Hierarchy : 100%|██████████| 196/196 [00:00<00:00, 17100.73it/s]


## Loading regions

In [12]:
%%capture
reg = timeseries.Region(("enzo", "Density"), resolution=(100, 100, 100))
timeseries.add_to_viewer(v, reg, file_pattern=file_pattern, load_as_stack=True, 
                         use_dask=True, 
                         contrast_limits=(-1, 2))

Parsing Hierarchy : 100%|██████████| 194/194 [00:00<00:00, 1804.52it/s]


## Using dask, returning in-memory image array 

We can also use dask to simply distribute the selection but still return a fully in-memory array. This works best for slices, where you **probably** can safely fit all those slices in memory. 

In [20]:
%%time
slice = timeseries.Slice(("enzo", "Density"), "x", resolution=(1600, 1600))
timeseries.add_to_viewer(v, slice, file_pattern=file_pattern, load_as_stack=True, 
                         use_dask=True, 
                         return_delayed = False,
                         contrast_limits=(-1, 2))

Parsing Hierarchy : 100%|██████████| 54/54 [00:00<00:00, 5102.67it/s]

Parsing Hierarchy : 100%|██████████| 2/2 [00:00<00:00, 16384.00it/s]

Parsing Hierarchy : 100%|██████████| 99/99 [00:00<00:00, 19060.64it/s]

Parsing Hierarchy : 100%|██████████| 21/21 [00:00<00:00, 11664.73it/s]


CPU times: user 1 s, sys: 869 ms, total: 1.87 s
Wall time: 12.3 s


Note that changing `return_delayed` to `False` with a region selection can quickly exceed the memory available on your machine! 

In [13]:
c.close()